# Introduction to the PL/pgSQL exit statement

The `exit` statement allows you to prematurely terminate a loop, including an **unconditional loop**, a **while loop**, and a **for loop**.

```postgresql
exit [label] [when boolean_expression] 
```

In this syntax:
* The `label` is the **loop label** of the current loop where the `exit` is in, or the **loop label** of the outer loop.
* Depending on the **label**, the `exit` statement will terminate the corresponding loop.
* If you don’t use the **label**, the `exit` statement will terminate the enclosing loop.
* Use the `when boolean_expression` clause to specify a condition that terminates a loop.
* The `exit` statement will terminate the loop if the `boolean_expression` evaluates to `true`.

The following statements are equivalent:

```postgresql
exit when counter > 10; 
```

```postgresql
if counter > 10 then
   exit;
end if; 
```

The `exit when` is cleaner and shorter.

Besides terminating a loop, you can use the `exit` statement to exit a block specified by the `begin...end` keywords.

In this case, the control is passed to the statement after the `end` keyword of the current block:

```postgresql
<<block_label>>
BEGIN
    -- some code
    EXIT [block_label] [WHEN condition];
    -- some more code
END block_label; 
```


# 1) Using PL/pgSQL Exit statement to terminate an unconditional loop

**Example 1:**

```postgresql
do
$$
declare
   i int = 0;
   j int = 0;
begin
  <<outer_loop>>
  loop
     i = i + 1;
     exit when i > 3;
	 -- inner loop
	 j = 0;
     <<inner_loop>>
     loop
		j = j + 1;
		exit when j > 3;
		raise notice '(i,j): (%,%)', i, j;
	 end loop inner_loop;
  end loop outer_loop;
end;
$$ 
```

**Output:**

```
NOTICE:  (i,j): (1,1)
NOTICE:  (i,j): (1,2)
NOTICE:  (i,j): (1,3)
NOTICE:  (i,j): (2,1)
NOTICE:  (i,j): (2,2)
NOTICE:  (i,j): (2,3)
NOTICE:  (i,j): (3,1)
NOTICE:  (i,j): (3,2)
NOTICE:  (i,j): (3,3)
```

**Example 2:**

```postgresql
do
$$
declare
   i int = 0;
   j int = 0;
begin
  <<outer_loop>>
  loop
     i = i + 1;
     exit when i > 3;
	 -- inner loop
	 j = 0;
     <<inner_loop>>
     loop
		j = j + 1;
		exit outer_loop when j > 3;
		raise notice '(i,j): (%,%)', i, j;
	 end loop inner_loop;
  end loop outer_loop;
end;
$$ 
```

**Output:**

```
NOTICE:  (i,j): (1,1)
NOTICE:  (i,j): (1,2)
NOTICE:  (i,j): (1,3)
```

# 2) Using the PL/pgSQL Exit statement to exit a block

```postgresql
do
$$
begin

  <<simple_block>>
   begin
  	 exit simple_block;
         -- for demo purposes
	 raise notice '%', 'unreachable!';
   end;
   raise notice '%', 'End of block';
end;
$$ 
```

**Output:**

```
NOTICE:  End of block
```

In this example, the `exit` statement terminates the `simple_block` immediately:

```postgresql
exit simple_block; 
```

This statement will never be reached:

```postgresql
raise notice '%', 'unreachable!'; 
```